# Debug

## Definitions

In [2]:
# impose modules, python 3.5
from sympy import symbols, cos, sin, pi, simplify, atan2, sqrt
from sympy.matrices import Matrix, matrix_multiply_elementwise
import numpy as np

In [2]:
# define symbols
q1, q2, q3, q4, q5, q6 = symbols('q1:7')

In [35]:
# alpha0, alpha1, alpha2, alpha3, alpha4, alpha5, alpha6 = symbols('alpha0:7')
# a0, a1, a2, a3, a4, a5, a6 = symbols('a0:7')
# d1, d2, d3, d4, d5, d6, dG = symbols('d1:8')
# theta1, theta2, theta3, theta4, theta5, theta6, thetaG = symbols('theta1:8')
# q1, q2, q3, q4, q5, q6 = symbols('q1:7')

In [3]:
# identity matrix
I = Matrix([[1, 0, 0], 
            [0, 1, 0], 
            [0, 0, 1]])

In [46]:
Pi = np.pi

In [45]:
rtd = 180. / Pi
dtr = Pi / 180.

In [3]:
# rotation matrices

def rot_x(q):
    R_x = Matrix([[ 1,              0,        0],
                  [ 0,         cos(q),  -sin(q)],
                  [ 0,         sin(q),  cos(q)]])
    
    return R_x
    
def rot_y(q):              
    R_y = Matrix([[ cos(q),        0,  sin(q)],
                  [      0,        1,       0],
                  [-sin(q),        0, cos(q)]])
    
    return R_y

def rot_z(q):    
    R_z = Matrix([[ cos(q),  -sin(q),       0],
                  [ sin(q),   cos(q),       0],
                  [      0,        0,       1]])
    
    return R_z

In [5]:
# translation vectors

def p_x(d):
    t = Matrix([[d], [0], [0]])
    return t

def p_y(d):
    t = Matrix([[0], [d], [0]])
    return t

def p_z(d):
    t = Matrix([[0], [0], [d]])
    return t

In [6]:
# homogeneous transform: put rotation and translation together
def H(R, p=Matrix([[0], [0], [0]])):
    '''
    R is rotation matrix
    p is translation vector
    '''
    D = R.row_join(p)
    D = D.col_join(Matrix([[0,0,0,1]]))
    return D

In [10]:
# homogeneous transform between neighbouring frames
def T(alpha, a, theta, d):
    T = H(rot_x(alpha)) * H(I, p_x(a)) * H(rot_z(theta)) * H(I, p_z(d))
    return T

In [11]:
# DH parameter
s = {'alpha0': 0,     'a0': 0,      'd1': 0.75,  'theta1': q1,
     'alpha1': -pi/2, 'a1': 0.35,   'd2': 0,     'theta2': q2 - pi/2,
     'alpha2': 0,     'a2': 1.25,   'd3': 0,     'theta3': q3,
     'alpha3': -pi/2, 'a3': -0.054, 'd4': 1.5,   'theta4': q4,
     'alpha4': pi/2,  'a4': 0,      'd5': 0,     'theta5': q5,
     'alpha5': -pi/2, 'a5': 0,      'd6': 0,     'theta6': q6,
     'alpha6': 0,     'a6': 0,      'dG': 0.303, 'thetaG': 0}

In [12]:
# Homogeneous transform between frames
T01 = T(s['alpha0'], s['a0'], s['theta1'], s['d1'])
T12 = T(s['alpha1'], s['a1'], s['theta2'], s['d2'])
T23 = T(s['alpha2'], s['a2'], s['theta3'], s['d3'])
T34 = T(s['alpha3'], s['a3'], s['theta4'], s['d4'])
T45 = T(s['alpha4'], s['a4'], s['theta5'], s['d5'])
T56 = T(s['alpha5'], s['a5'], s['theta6'], s['d6'])
T6G = T(s['alpha6'], s['a6'], s['thetaG'], s['dG'])

In [18]:
# transform from base_point to G
T0G = T01 * T12 * T23 * T34 * T45 * T56 * T6G

In [13]:
# initial state of joints
state_i = {q1: 0, q2: 0, q3: 0, q4: 0, q5: 0, q6: 0}

In [14]:
# origin of frame
origin = Matrix([[0,0,0,1]]).T

## Forward Kinematics

In [17]:
# frame G origin
G = origin

In [28]:
# forward kinematics result
pOG = T0G * G

In [30]:
def FK_check(state, pG):
    pG_na = np.array(pG).astype(np.float32)
    delta_pG = (pOG - pG).evalf(subs=state)
    delta_pG_na = np.array(delta_pG).astype(np.float32)
    print('The position of G is\n')
    print(pG_na)
    print('\n')
    print('The forward kinematics position of G is \n')
    print(pOG.evalf(subs=state))
    print('\n')
    print('The difference between two position is\n')
    print(delta_pG)
    print('\n The difference ratio is')
    print(delta_pG_na/pG_na)

In [31]:
# random joint state 1
state1 = {q1: 0.31, q2: 0.83, q3: 1.05, q4: -4.24, q5: 1.46, q6: 5.54}

# random gripper position 1
pG1 = Matrix([[0.75297],[0.52647],[0.10997],[1]])

In [32]:
FK_check(state1, pG1)

The position of G is

[[ 0.75296998]
 [ 0.52647001]
 [ 0.10997   ]
 [ 1.        ]]


The forward kinematics position of G is 

Matrix([[0.760872822196658], [0.525312787626134], [0.107552322756199], [1.00000000000000]])


The difference between two position is

Matrix([[0.00790282219665779], [-0.00115721237386610], [-0.00241767724380139], [0]])

 The difference ratio is
[[ 0.01049553]
 [-0.00219806]
 [-0.02198488]
 [ 0.        ]]


In [33]:
# random joint state 2
state2 = {q1: 1.65, q2: 0.02, q3: -0.48, q4: 0.81, q5: -1.38, q6: -0.25}

# random gripper position 1
pG2 = Matrix([[0.074248],[1.7157],[2.8274],[1]])

In [34]:
FK_check(state2, pG2)

The position of G is

[[ 0.074248  ]
 [ 1.71570003]
 [ 2.82739997]
 [ 1.        ]]


The forward kinematics position of G is 

Matrix([[0.0800208120974468], [1.71518116673536], [2.82659958397620], [1.00000000000000]])


The difference between two position is

Matrix([[0.00577281209744679], [-0.000518833264635179], [-0.000800416023799421], [0]])

 The difference ratio is
[[ 0.07775041]
 [-0.0003024 ]
 [-0.00028309]
 [ 0.        ]]


## Inverse Kinematics

In [1]:
import tf

In [6]:
roll, pitch, yaw = tf.transformations.euler_from_quaternion([0.7068, -0.27077, 0.25973, 0.59972])
print(roll, pitch, yaw)

AttributeError: module 'tf' has no attribute 'transformations'

In [4]:
R = rot_z(yaw) * rot_y(pitch) * rot_x(roll)

NameError: name 'yaw' is not defined

In [24]:
pG1_na = 
print(pG1_na)

[[ 0.75296998]
 [ 0.52647001]
 [ 0.10997   ]
 [ 1.        ]]


In [84]:
print(T01.evalf(subs={q1:0}))

Matrix([[1.00000000000000, 0, 0, 0], [0, 1.00000000000000, 0, 0], [0, 0, 1.00000000000000, 0.750000000000000], [0, 0, 0, 1.00000000000000]])


In [87]:
OG = T0G * G

In [88]:
state_i = {q1: 0, q2: 0, q3: 0, q4: 0, q5: 0, q6: 0}

In [90]:
print(OG.evalf(subs=state_i))

Matrix([[2.15300000000000], [0], [1.94600000000000], [1.00000000000000]])


In [107]:
state = {q1: 0, q2: 0, q3: 0, q4: 0, q5: 0, q6: 2.58}

In [108]:
print(OG.evalf(subs=state))

Matrix([[2.15300000000000], [0], [1.94600000000000], [1.00000000000000]])


In [109]:
T24 = T23 * T34
print(T24.evalf())

Matrix([[cos(q3)*cos(q4), -sin(q4)*cos(q3), -sin(q3), -1.5*sin(q3) - 0.054*cos(q3) + 1.25], [sin(q3)*cos(q4), -sin(q3)*sin(q4), cos(q3), -0.054*sin(q3) + 1.5*cos(q3)], [-sin(q4), -cos(q4), 0, 0], [0, 0, 0, 1.00000000000000]])


In [111]:
T04 = T01 * T12 * T23 * T34
print((simplify(T04 * G)).evalf())

Matrix([
[(1.25*sin(q2) - 0.054*sin(q2 + q3) + 1.5*cos(q2 + q3) + 0.35)*cos(q1)],
[(1.25*sin(q2) - 0.054*sin(q2 + q3) + 1.5*cos(q2 + q3) + 0.35)*sin(q1)],
[         -1.5*sin(q2 + q3) + 1.25*cos(q2) - 0.054*cos(q2 + q3) + 0.75],
[                                                                  1.0]])


In [112]:
T14 = T12 * T23 * T34
print((simplify(T14 * G)).evalf())

Matrix([
[1.25*sin(q2) - 0.054*sin(q2 + q3) + 1.5*cos(q2 + q3) + 0.35],
[                                                          0],
[      -1.5*sin(q2 + q3) + 1.25*cos(q2) - 0.054*cos(q2 + q3)],
[                                                        1.0]])


In [113]:
T36 = T34 * T45 * T56
print((simplify(T36)).evalf())

Matrix([
[-sin(q4)*sin(q6) + cos(q4)*cos(q5)*cos(q6), -sin(q4)*cos(q6) - sin(q6)*cos(q4)*cos(q5), -sin(q5)*cos(q4), -0.054],
[                           sin(q5)*cos(q6),                           -sin(q5)*sin(q6),          cos(q5),    1.5],
[-sin(q4)*cos(q5)*cos(q6) - sin(q6)*cos(q4),  sin(q4)*sin(q6)*cos(q5) - cos(q4)*cos(q6),  sin(q4)*sin(q5),      0],
[                                         0,                                          0,                0,    1.0]])


In [11]:
state_i = {alpha0: 0,     a0: 0,      d1: 0.75,  theta1: 0,
         alpha1: -pi/2, a1: 0.35,   d2: 0,     theta2: -pi/2,
         alpha2: 0,     a2: 1.25,   d3: 0,     theta3: 0, 
         alpha3: -pi/2, a3: -0.054, d4: 1.5,   theta4: 0, 
         alpha4: pi/2,  a4: 0,      d5: 0,     theta5: 0, 
         alpha5: -pi/2, a5: 0,      d6: 0,     theta6: 0,
         alpha6: 0,     a6: 0,      dG: 0.303, thetaG: 0}

In [13]:
OG = T0G * O.T

In [17]:
Ti = [T01, T12, T23, T34, T45, T56, T6G]

In [22]:
c1, c2, c3 = symbols('c1:4')

In [31]:
c = c1 + c2
c = c.evalf(subs={c1:1})
print(c)

c1 + c2


In [21]:
for t in Ti:
    print('\n', t.evalf(subs=state))


 Matrix([[cos(theta1), -sin(theta1), 0, 0], [sin(theta1)*cos(alpha0), cos(alpha0)*cos(theta1), 0, 0], [sin(alpha0)*sin(theta1), sin(alpha0)*cos(theta1), 1.00000000000000, 0.750000000000000], [0, 0, 0, 1.00000000000000]])

 Matrix([[cos(theta2), -sin(theta2), 0, 0.350000000000000], [sin(theta2)*cos(alpha1), cos(alpha1)*cos(theta2), 1.00000000000000, 0], [sin(alpha1)*sin(theta2), sin(alpha1)*cos(theta2), -0.e-189, 0], [0, 0, 0, 1.00000000000000]])

 Matrix([[cos(theta3), -sin(theta3), 0, 1.25000000000000], [sin(theta3)*cos(alpha2), cos(alpha2)*cos(theta3), 0, 0], [sin(alpha2)*sin(theta3), sin(alpha2)*cos(theta3), 1.00000000000000, 0], [0, 0, 0, 1.00000000000000]])

 Matrix([[cos(theta4), -sin(theta4), 0, -0.0540000000000000], [sin(theta4)*cos(alpha3), cos(alpha3)*cos(theta4), 1.00000000000000, 1.50000000000000], [sin(alpha3)*sin(theta4), sin(alpha3)*cos(theta4), -0.e-189, -0.e-214], [0, 0, 0, 1.00000000000000]])

 Matrix([[cos(theta5), -sin(theta5), 0, 0], [sin(theta5)*cos(alpha4), cos(

In [16]:
print(T12.evalf(subs=state_i))

Matrix([[-0.e-189, 1.00000000000000, 0, 0.350000000000000], [0.e-214, 0.e-428, 1.00000000000000, 0], [1.00000000000000, 0.e-214, -0.e-189, 0], [0, 0, 0, 1.00000000000000]])


In [25]:
print(OG.evalf(subs=state_i))

KeyboardInterrupt: 

In [18]:
O3 = T03

In [20]:
print(T01.evalf(subs=state))

Matrix([[1.00000000000000, 0, 0, 0], [0, 1.00000000000000, 0, 0], [0, 0, 1.00000000000000, 0.750000000000000], [0, 0, 0, 1.00000000000000]])


In [22]:
print(T01.T.evalf(subs=state))

Matrix([[1.00000000000000, 0, 0, 0], [0, 1.00000000000000, 0, 0], [0, 0, 1.00000000000000, 0], [0, 0, 0.750000000000000, 1.00000000000000]])
